In [34]:
import pandas as pd
import os

In [35]:
"""
Fin del notebook: exportar una tabla que contenga todos los genomas de importancia clinica en la cantidad
de veces que posean un AMR de importancia
Además tabla contiene la cantidad replicones 
Producto: Una tabla con todos los genomas que tienen un AMR, replicones, nicho y año de aislamiento
hay 6 serotipos de importancia, con los AMR de importancia y los replicones positivos
*La agrupación de los AMR y replicones es por tener mas del 80 %, en total de todos los genomas del serotipo 
"""
db = pd.read_csv(os.path.join("out", "amr.tsv"), sep='\t')
db = db.drop(columns=['HC20', 'HC50', 'SISTR1 Serovar HC20', 'SISTR1 Serovar HC50'])
db

,Assembly Barcode,Resistance gene,Class,SISTR1 Serovar,Source Niche,Collection Year
0,SAL_AD8849AA_AS,floR,PHENICOL,Adelaide,Environment,2022
1,SAL_AD8849AA_AS,blaTEM-1,BETA-LACTAM,Adelaide,Environment,2022
2,SAL_AD8849AA_AS,dfrA12,TRIMETHOPRIM,Adelaide,Environment,2022
3,SAL_AD8849AA_AS,aadA2,AMINOGLYCOSIDE,Adelaide,Environment,2022
4,SAL_AD8849AA_AS,sul1,SULFONAMIDE,Adelaide,Environment,2022
...,...,...,...,...,...,...
2150,SAL_DC0019AA_AS,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2019
2151,SAL_DC0019AA_AS,sul1,SULFONAMIDE,Typhimurium,Environment,2019
2152,SAL_DC0019AA_AS,aadA2,AMINOGLYCOSIDE,Typhimurium,Environment,2019
2153,SAL_OC0804AA_AS,tet(G),TETRACYCLINE,Typhimurium,Environment,2019


In [36]:
FAMILIAS = ['QUINOLONE', 'BETA-LACTAM', 'MACROLIDE', 'TRIMETHOPRIM']
SEROTIPOS = ['Typhimurium', 'Newport', 'Enteritidis', 'Infantis', 'I 1,4,[5],12:i:-', 'Agona']

In [37]:
amr = db[db['Class'].isin(FAMILIAS)]
print(amr.shape[0])
amr = amr[amr['SISTR1 Serovar'].isin(SEROTIPOS)]
print(amr.shape[0])
amr = amr.set_index('Assembly Barcode')
amr

598
524


,Resistance gene,Class,SISTR1 Serovar,Source Niche,Collection Year
Assembly Barcode,,,,,
SAL_QD2199AA_AS,blaTEM-1,BETA-LACTAM,Agona,Environment,2022
SAL_QD2199AA_AS,qnrS1,QUINOLONE,Agona,Environment,2022
SAL_ZC7016AA_AS,dfrA17,TRIMETHOPRIM,Agona,Companion Animal,2021
SAL_JC3732AA_AS,blaTEM-1,BETA-LACTAM,Agona,Food,2018
SAL_DC0057AA_AS,blaTEM-1,BETA-LACTAM,Agona,Environment,2019
...,...,...,...,...,...
SAL_MB2490AA_AS,blaCARB-2,BETA-LACTAM,Typhimurium,Livestock,2016
SAL_MB2476AA_AS,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2016
SAL_MB2470AA_AS,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2015


In [38]:
amr_matrix = amr.groupby([f'SISTR1 Serovar', 'Resistance gene']).size().unstack(fill_value=0)
amr_matrix

Resistance gene,blaCARB-2,blaCTX-M-65,blaROB-11,blaTEM-1,dfrA1,dfrA14,dfrA17,dfrA8,estT,gyrA_D87Y,gyrA_S83Y,gyrB_S464Y,mph(A),qnrA1,qnrB,qnrB19,qnrS1
SISTR1 Serovar,,,,,,,,,,,,,,,,,
Agona,0,0,0,3,0,2,1,0,0,0,0,0,0,0,0,0,1
Enteritidis,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0
"I 1,4,[5],12:i:-",0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,1,0
Infantis,0,131,0,2,0,129,0,2,0,198,0,2,0,0,0,3,0
Newport,2,0,2,2,2,0,0,0,2,0,0,0,2,2,0,0,0
Typhimurium,8,0,0,5,0,1,0,2,0,0,3,0,0,0,1,7,1


In [39]:
heat_map = amr_matrix.apply(lambda x: ((x - x.min()) / (x.max() - x.min()) * 100).astype(int), axis=1)
heat_map

Resistance gene,blaCARB-2,blaCTX-M-65,blaROB-11,blaTEM-1,dfrA1,dfrA14,dfrA17,dfrA8,estT,gyrA_D87Y,gyrA_S83Y,gyrB_S464Y,mph(A),qnrA1,qnrB,qnrB19,qnrS1
SISTR1 Serovar,,,,,,,,,,,,,,,,,
Agona,0,0,0,100,0,66,33,0,0,0,0,0,0,0,0,0,33
Enteritidis,0,0,0,100,0,0,0,0,0,100,100,0,0,0,0,100,0
"I 1,4,[5],12:i:-",0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,33,0
Infantis,0,66,0,1,0,65,0,1,0,100,0,1,0,0,0,1,0
Newport,100,0,100,100,100,0,0,0,100,0,0,0,100,100,0,0,0
Typhimurium,100,0,0,62,0,12,0,25,0,0,37,0,0,0,12,87,12


In [40]:
"""
Un json con llave el serotipo de importancia, value, una lista con los AMR sobre el 80 %
de las familias de importancia
"""
AMR_sobre_80 = {}
for index, row in heat_map.iterrows():
    sobre_80 = row[row > 80].index.tolist()
    AMR_sobre_80[index] = sobre_80

rows = [(serotype, amr) for serotype, amrs in AMR_sobre_80.items() for amr in amrs]
AMR_sobre_80 = pd.DataFrame(rows, columns=['SISTR1 Serovar', 'Resistance gene'])
AMR_sobre_80

,SISTR1 Serovar,Resistance gene
0,Agona,blaTEM-1
1,Enteritidis,blaTEM-1
2,Enteritidis,gyrA_D87Y
3,Enteritidis,gyrA_S83Y
4,Enteritidis,qnrB19
5,"I 1,4,[5],12:i:-",blaTEM-1
6,Infantis,gyrA_D87Y
7,Newport,blaCARB-2
8,Newport,blaROB-11
9,Newport,blaTEM-1


### Tabla con todos los AMR y serotipos de importancia

In [41]:
amr_filtrado = amr.merge(AMR_sobre_80, on=['SISTR1 Serovar', 'Resistance gene'])
amr_filtrado

,Resistance gene,Class,SISTR1 Serovar,Source Niche,Collection Year
0,blaTEM-1,BETA-LACTAM,Agona,Environment,2022
1,blaTEM-1,BETA-LACTAM,Agona,Food,2018
2,blaTEM-1,BETA-LACTAM,Agona,Environment,2019
3,gyrA_D87Y,QUINOLONE,Enteritidis,Environment,2021
4,blaTEM-1,BETA-LACTAM,Enteritidis,Poultry,2023
...,...,...,...,...,...
232,blaCARB-2,BETA-LACTAM,Typhimurium,Livestock,2016
233,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2016
234,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2015
235,blaCARB-2,BETA-LACTAM,Typhimurium,Environment,2022


#### Sacar replicones de los serotipos de importancia 

In [42]:
abricate_db = pd.read_csv(os.path.join("out", "abricate_summary_procesado.tsv"), sep='\t')
abricate_db = abricate_db.drop(columns=['SISTR1 Serovar HC20', 'SISTR1 Serovar HC50'])
abricate_db

,Col(MG828)_1,Col(VCM04)_1,Col(Ye4449)_1,Col(pB8919)_1,Col(pHAD28)_1,Col156_1,Col3M_1,Col440II_1,Col440I_1,ColE10_1,...,IncX4_1,IncX4_2,IncY_1,p0111_1,pEC4115_1,pESA2_1,pKPC-CAV1321_1,repB(R1701)_1,repE(pEh60-7)_1,SISTR1 Serovar
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Senftenberg
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Enteritidis
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Agona
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Typhimurium
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"I 1,4,[5],12:i:-"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Infantis
654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Infantis
655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Infantis
656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Infantis


In [43]:
abricate_db_filtrada = abricate_db[abricate_db['SISTR1 Serovar'].isin(SEROTIPOS)]
abricate_matrix = abricate_db_filtrada.groupby(f'SISTR1 Serovar').sum()
abricate_matrix

,Col(MG828)_1,Col(VCM04)_1,Col(Ye4449)_1,Col(pB8919)_1,Col(pHAD28)_1,Col156_1,Col3M_1,Col440II_1,Col440I_1,ColE10_1,...,IncX3_1,IncX4_1,IncX4_2,IncY_1,p0111_1,pEC4115_1,pESA2_1,pKPC-CAV1321_1,repB(R1701)_1,repE(pEh60-7)_1
SISTR1 Serovar,,,,,,,,,,,,,,,,,,,,,
Agona,0,0,0,0,3,2,3,2,0,0,...,0,0,0,0,2,0,2,0,0,0
Enteritidis,0,0,0,0,4,1,0,0,0,0,...,22,0,0,0,0,0,0,0,0,0
"I 1,4,[5],12:i:-",0,0,0,0,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Infantis,1,0,0,0,3,1,0,0,0,0,...,1,0,8,0,0,2,0,0,0,0
Newport,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Typhimurium,0,4,0,0,21,12,0,8,0,0,...,3,2,9,0,0,0,0,0,0,0


In [44]:
heat_map = abricate_matrix.apply(lambda x: ((x - x.min()) / (x.max() - x.min()) * 100).astype(int), axis=1)
heat_map

,Col(MG828)_1,Col(VCM04)_1,Col(Ye4449)_1,Col(pB8919)_1,Col(pHAD28)_1,Col156_1,Col3M_1,Col440II_1,Col440I_1,ColE10_1,...,IncX3_1,IncX4_1,IncX4_2,IncY_1,p0111_1,pEC4115_1,pESA2_1,pKPC-CAV1321_1,repB(R1701)_1,repE(pEh60-7)_1
SISTR1 Serovar,,,,,,,,,,,,,,,,,,,,,
Agona,0,0,0,0,100,66,100,66,0,0,...,0,0,0,0,66,0,66,0,0,0
Enteritidis,0,0,0,0,5,1,0,0,0,0,...,27,0,0,0,0,0,0,0,0,0
"I 1,4,[5],12:i:-",0,0,0,0,100,50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Infantis,0,0,0,0,1,0,0,0,0,0,...,0,0,4,0,0,1,0,0,0,0
Newport,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Typhimurium,0,4,0,0,25,14,0,9,0,0,...,3,2,11,0,0,0,0,0,0,0


In [45]:
"""
Un json con llave el serotipo de importancia, value, una lista con los replicones sobre el 80 %
de las familias de importancia
"""
replicones_sobre_80 = {}
for index, row in heat_map.iterrows():
    sobre_80 = row[row > 80].index.tolist()
    replicones_sobre_80[index] = sobre_80

rows = [(serotype, plasmid) for serotype, plasmids in replicones_sobre_80.items() for plasmid in plasmids]
replicones_sobre_80 = pd.DataFrame(rows, columns=['SISTR1 Serovar', 'Replicon'])
replicones_sobre_80

,SISTR1 Serovar,Replicon
0,Agona,Col(pHAD28)_1
1,Agona,Col3M_1
2,Agona,ColRNAI_1
3,Agona,IncFIB(K)_1
4,Agona,IncI1-I(Alpha)_1
5,Enteritidis,IncFIB(S)_1
6,Enteritidis,IncFII(S)_1
7,Enteritidis,IncFII(pAR0022)_1
8,"I 1,4,[5],12:i:-",Col(pHAD28)_1
9,"I 1,4,[5],12:i:-",IncFIB(S)_1


In [46]:
"""
f1: implica el 80 de todos los AMR para el serotipo, se omite el HC 
tanto para el AMR como para los replicones
"""
tabla_total = amr_filtrado.merge(replicones_sobre_80, on='SISTR1 Serovar', how='left')

tabla_total = tabla_total.rename(columns={'Resistance gene': 'AMR', 'SISTR1 Serovar': 'Serovar', 'Source Niche': 'Niche', 'Collection Year': 'Year'})
tabla_total = tabla_total.drop(columns=['AMR'])

tabla_total.to_csv("out/mca_tabla_f1.csv", index=False)
tabla_total

,Class,Serovar,Niche,Year,Replicon
0,BETA-LACTAM,Agona,Environment,2022,Col(pHAD28)_1
1,BETA-LACTAM,Agona,Environment,2022,Col3M_1
2,BETA-LACTAM,Agona,Environment,2022,ColRNAI_1
3,BETA-LACTAM,Agona,Environment,2022,IncFIB(K)_1
4,BETA-LACTAM,Agona,Environment,2022,IncI1-I(Alpha)_1
...,...,...,...,...,...
291,BETA-LACTAM,Typhimurium,Environment,2022,IncFII(S)_1
292,BETA-LACTAM,Typhimurium,Environment,2022,IncFII(pAR0022)_1
293,BETA-LACTAM,Typhimurium,Environment,2019,IncFIB(S)_1
294,BETA-LACTAM,Typhimurium,Environment,2019,IncFII(S)_1
